In [1]:
# =============================================================
# ADVANCED MULTI-SPRINKLER HYBRID SYSTEM
# BDI + DELIBERATIVE + MODEL-BASED REFLEX + LEARNING
# (Evaporation + Forward Temporal Awareness FIXED)
# =============================================================

import random
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_text

# -----------------------------
# PARAMETERS
# -----------------------------
SPRINKLERS = ["Sprklr1", "Sprklr2", "Sprklr3"]
TARGET = 65
LOWER_BUFFER = 60   # Prevent collapse below this
EPISODES = 150

PRESSURE_MODES = ["OFF", "LOW", "HIGH"]
CYCLE_TIMES = [8, 12, 16, 20]
MAX_CYCLES = 4

weather_states = ["snow", "rain", "humid", "hot"]
MODELS = {}

# -----------------------------
# WEATHER + DYNAMICS
# -----------------------------
def base_weather(weather):
    if weather == "hot":
        return 35, 30
    elif weather == "humid":
        return 28, 80
    elif weather == "rain":
        return 24, 90
    elif weather == "snow":
        return 2, 70
    else:
        return 30, 50

def dynamic_temperature(base_temp, hour):
    if hour <= 1 or hour >= 20:
        return base_temp + 2
    elif 1 < hour <= 13:
        return base_temp + (13 - hour)
    else:
        return base_temp

def evaporation_rate(temp, humidity):
    return max(1, (temp / 6) - (humidity / 100))

def get_state(moisture):
    return int(moisture // 5) * 5

def step(moisture, weather, pressure, hour):

    base_temp, humidity = base_weather(weather)
    temp = dynamic_temperature(base_temp, hour)
    evap = evaporation_rate(temp, humidity)

    if weather in ["rain", "snow"]:
        pressure = "OFF"

    if pressure == "OFF":
        water = 0
        cost = 0
    elif pressure == "LOW":
        water = 8
        cost = 1
    else:
        water = 15
        cost = 3

    new_moisture = moisture + water - evap
    new_moisture = max(0, min(100, new_moisture))

    reward = -abs(TARGET - new_moisture) - cost

    return new_moisture, reward, temp, evap


# =============================================================
# TRAINING PHASE (FORWARD AWARE)
# =============================================================

print("\n================ TRAINING (FORWARD-AWARE) =================")

for s in SPRINKLERS:

    X = []
    y = []

    for ep in range(EPISODES):

        weather = random.choice(weather_states)
        weather_index = weather_states.index(weather)
        moisture = random.randint(40, 80)

        for cycle in range(MAX_CYCLES):

            hour = CYCLE_TIMES[cycle]
            base_temp, humidity = base_weather(weather)
            temp = dynamic_temperature(base_temp, hour)
            evap = evaporation_rate(temp, humidity)

            state = get_state(moisture)

            # Predict future moisture if we do NOTHING
            future_moisture, _, _, _ = step(moisture, weather, "OFF", hour)

            # SMART POLICY (Forward aware)
            if moisture > TARGET:
                if future_moisture < LOWER_BUFFER:
                    best_action = "LOW"
                else:
                    best_action = "OFF"
            elif moisture < LOWER_BUFFER:
                best_action = "HIGH"
            else:
                best_action = "LOW"

            X.append([
                state,
                weather_index,
                hour,
                round(temp,2),
                round(evap,2),
                round(future_moisture,2)
            ])

            y.append(PRESSURE_MODES.index(best_action))

            moisture, _, _, _ = step(moisture, weather, best_action, hour)

    model = DecisionTreeClassifier(
        criterion="entropy",
        max_depth=6
    )

    model.fit(X, y)
    MODELS[s] = model

print("\nTraining Complete.\n")

print("===== Sample Decision Tree (Sprklr1) =====")
print(export_text(
    MODELS["Sprklr1"],
    feature_names=["State","Weather","Hour","Temp","Evap","FutureMoisture"]
))

# =============================================================
# USER PHASE
# =============================================================

moisture_levels = {s: 70 for s in SPRINKLERS}

for day in range(5):

    weather = input(f"\nEnter weather for Day {day+1} (hot/humid/rain/snow): ").lower()
    weather_index = weather_states.index(weather)

    print("\n===================================================")
    print(f"DAY {day+1} | Weather: {weather}")
    print("Start Moisture:", moisture_levels)

    if weather in ["rain","snow"]:
        print("System OFF due to weather.")
        continue

    for cycle in range(MAX_CYCLES):

        hour = CYCLE_TIMES[cycle]
        print(f"\n--- Cycle {cycle+1} at {hour}:00 ---")

        for s in SPRINKLERS:

            base_temp, humidity = base_weather(weather)
            temp = dynamic_temperature(base_temp, hour)
            evap = evaporation_rate(temp, humidity)

            state = get_state(moisture_levels[s])
            future_m, _, _, _ = step(moisture_levels[s], weather, "OFF", hour)

            features = [[
                state,
                weather_index,
                hour,
                round(temp,2),
                round(evap,2),
                round(future_m,2)
            ]]

            action_index = MODELS[s].predict(features)[0]
            action = PRESSURE_MODES[action_index]

            print(f"\n{s}")
            print(f"  Current Moisture: {round(moisture_levels[s],2)}")
            print(f"  Future Moisture if OFF: {round(future_m,2)}")
            print(f"  Decision Tree Action: {action}")

            new_m, _, _, _ = step(moisture_levels[s], weather, action, hour)

            print(f"  Moisture After Cycle: {round(new_m,2)}")

            moisture_levels[s] = new_m

    print("\nEnd of Day Moisture:", moisture_levels)

print("\n================ END SIMULATION ================")


================ TRAINING (FORWARD-AWARE) =================

Training Complete.

===== Sample Decision Tree (Sprklr1) =====
|--- State <= 57.50
|   |--- class: 2
|--- State >  57.50
|   |--- State <= 62.50
|   |   |--- class: 1
|   |--- State >  62.50
|   |   |--- FutureMoisture <= 60.17
|   |   |   |--- class: 1
|   |   |--- FutureMoisture >  60.17
|   |   |   |--- Weather <= 1.50
|   |   |   |   |--- FutureMoisture <= 64.13
|   |   |   |   |   |--- FutureMoisture <= 63.90
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |   |--- FutureMoisture >  63.90
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |--- FutureMoisture >  64.13
|   |   |   |   |   |--- class: 0
|   |   |   |--- Weather >  1.50
|   |   |   |   |--- class: 0




Enter weather for Day 1 (hot/humid/rain/snow):  humid



DAY 1 | Weather: humid
Start Moisture: {'Sprklr1': 70, 'Sprklr2': 70, 'Sprklr3': 70}

--- Cycle 1 at 8:00 ---

Sprklr1
  Current Moisture: 70
  Future Moisture if OFF: 65.3
  Decision Tree Action: OFF
  Moisture After Cycle: 65.3

Sprklr2
  Current Moisture: 70
  Future Moisture if OFF: 65.3
  Decision Tree Action: OFF
  Moisture After Cycle: 65.3

Sprklr3
  Current Moisture: 70
  Future Moisture if OFF: 65.3
  Decision Tree Action: OFF
  Moisture After Cycle: 65.3

--- Cycle 2 at 12:00 ---

Sprklr1
  Current Moisture: 65.3
  Future Moisture if OFF: 61.27
  Decision Tree Action: OFF
  Moisture After Cycle: 61.27

Sprklr2
  Current Moisture: 65.3
  Future Moisture if OFF: 61.27
  Decision Tree Action: OFF
  Moisture After Cycle: 61.27

Sprklr3
  Current Moisture: 65.3
  Future Moisture if OFF: 61.27
  Decision Tree Action: OFF
  Moisture After Cycle: 61.27

--- Cycle 3 at 16:00 ---

Sprklr1
  Current Moisture: 61.27
  Future Moisture if OFF: 57.4
  Decision Tree Action: LOW
  Moisture 


Enter weather for Day 2 (hot/humid/rain/snow):  hot



DAY 2 | Weather: hot
Start Moisture: {'Sprklr1': 61.2, 'Sprklr2': 61.2, 'Sprklr3': 61.2}

--- Cycle 1 at 8:00 ---

Sprklr1
  Current Moisture: 61.2
  Future Moisture if OFF: 54.83
  Decision Tree Action: LOW
  Moisture After Cycle: 62.83

Sprklr2
  Current Moisture: 61.2
  Future Moisture if OFF: 54.83
  Decision Tree Action: LOW
  Moisture After Cycle: 62.83

Sprklr3
  Current Moisture: 61.2
  Future Moisture if OFF: 54.83
  Decision Tree Action: LOW
  Moisture After Cycle: 62.83

--- Cycle 2 at 12:00 ---

Sprklr1
  Current Moisture: 62.83
  Future Moisture if OFF: 57.13
  Decision Tree Action: LOW
  Moisture After Cycle: 65.13

Sprklr2
  Current Moisture: 62.83
  Future Moisture if OFF: 57.13
  Decision Tree Action: LOW
  Moisture After Cycle: 65.13

Sprklr3
  Current Moisture: 62.83
  Future Moisture if OFF: 57.13
  Decision Tree Action: LOW
  Moisture After Cycle: 65.13

--- Cycle 3 at 16:00 ---

Sprklr1
  Current Moisture: 65.13
  Future Moisture if OFF: 59.6
  Decision Tree Acti

...


DAY 3 | Weather: hot
Start Moisture: {'Sprklr1': 61.73333333333334, 'Sprklr2': 61.73333333333334, 'Sprklr3': 61.73333333333334}

--- Cycle 1 at 8:00 ---

Sprklr1
  Current Moisture: 61.73
  Future Moisture if OFF: 55.37
  Decision Tree Action: LOW
  Moisture After Cycle: 63.37

Sprklr2
  Current Moisture: 61.73
  Future Moisture if OFF: 55.37
  Decision Tree Action: LOW
  Moisture After Cycle: 63.37

Sprklr3
  Current Moisture: 61.73
  Future Moisture if OFF: 55.37
  Decision Tree Action: LOW
  Moisture After Cycle: 63.37

--- Cycle 2 at 12:00 ---

Sprklr1
  Current Moisture: 63.37
  Future Moisture if OFF: 57.67
  Decision Tree Action: LOW
  Moisture After Cycle: 65.67

Sprklr2
  Current Moisture: 63.37
  Future Moisture if OFF: 57.67
  Decision Tree Action: LOW
  Moisture After Cycle: 65.67

Sprklr3
  Current Moisture: 63.37
  Future Moisture if OFF: 57.67
  Decision Tree Action: LOW
  Moisture After Cycle: 65.67

--- Cycle 3 at 16:00 ---

Sprklr1
  Current Moisture: 65.67
  Future 


Enter weather for Day 4 (hot/humid/rain/snow):  hot



DAY 4 | Weather: hot
Start Moisture: {'Sprklr1': 62.26666666666667, 'Sprklr2': 62.26666666666667, 'Sprklr3': 62.26666666666667}

--- Cycle 1 at 8:00 ---

Sprklr1
  Current Moisture: 62.27
  Future Moisture if OFF: 55.9
  Decision Tree Action: LOW
  Moisture After Cycle: 63.9

Sprklr2
  Current Moisture: 62.27
  Future Moisture if OFF: 55.9
  Decision Tree Action: LOW
  Moisture After Cycle: 63.9

Sprklr3
  Current Moisture: 62.27
  Future Moisture if OFF: 55.9
  Decision Tree Action: LOW
  Moisture After Cycle: 63.9

--- Cycle 2 at 12:00 ---

Sprklr1
  Current Moisture: 63.9
  Future Moisture if OFF: 58.2
  Decision Tree Action: LOW
  Moisture After Cycle: 66.2

Sprklr2
  Current Moisture: 63.9
  Future Moisture if OFF: 58.2
  Decision Tree Action: LOW
  Moisture After Cycle: 66.2

Sprklr3
  Current Moisture: 63.9
  Future Moisture if OFF: 58.2
  Decision Tree Action: LOW
  Moisture After Cycle: 66.2

--- Cycle 3 at 16:00 ---

Sprklr1
  Current Moisture: 66.2
  Future Moisture if OFF:


Enter weather for Day 5 (hot/humid/rain/snow):  humid



DAY 5 | Weather: humid
Start Moisture: {'Sprklr1': 62.800000000000004, 'Sprklr2': 62.800000000000004, 'Sprklr3': 62.800000000000004}

--- Cycle 1 at 8:00 ---

Sprklr1
  Current Moisture: 62.8
  Future Moisture if OFF: 58.1
  Decision Tree Action: LOW
  Moisture After Cycle: 66.1

Sprklr2
  Current Moisture: 62.8
  Future Moisture if OFF: 58.1
  Decision Tree Action: LOW
  Moisture After Cycle: 66.1

Sprklr3
  Current Moisture: 62.8
  Future Moisture if OFF: 58.1
  Decision Tree Action: LOW
  Moisture After Cycle: 66.1

--- Cycle 2 at 12:00 ---

Sprklr1
  Current Moisture: 66.1
  Future Moisture if OFF: 62.07
  Decision Tree Action: OFF
  Moisture After Cycle: 62.07

Sprklr2
  Current Moisture: 66.1
  Future Moisture if OFF: 62.07
  Decision Tree Action: OFF
  Moisture After Cycle: 62.07

Sprklr3
  Current Moisture: 66.1
  Future Moisture if OFF: 62.07
  Decision Tree Action: OFF
  Moisture After Cycle: 62.07

--- Cycle 3 at 16:00 ---

Sprklr1
  Current Moisture: 62.07
  Future Moistur